In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from datetime import datetime

# Load the first sheet (project details) and second sheet (delay reasons)
file_path = 'Insights INC. - Project Details - Dataset.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load both sheets
df1 = pd.read_excel(excel_data, sheet_name=0)  # First sheet (project details)
df2 = pd.read_excel(excel_data, sheet_name=1)  # Second sheet (delay reasons)

# Merge both dataframes on 'Project Code' (from df1) and 'Code' (from df2)
df = pd.merge(df1, df2, left_on='Project Code', right_on='Code', how='left')

# Clean column names
df.columns = df.columns.str.strip()  # Strip any leading/trailing spaces from column names

# Convert date columns to datetime format
df['Planned Project Start Date'] = pd.to_datetime(df['Planned Project Start Date'], errors='coerce')
df['Actual Project Start Date'] = pd.to_datetime(df['Actual Project Start Date'], errors='coerce')
df['Planned Go-Live Date'] = pd.to_datetime(df['Planned Go-Live Date'], errors='coerce')
df['Actual Go-Live Date'] = pd.to_datetime(df['Actual Go-Live Date'], errors='coerce')

# Calculate additional columns for analysis
df['Start Date Variance'] = (df['Actual Project Start Date'] - df['Planned Project Start Date']).dt.days
df['Go-Live Date Variance'] = (df['Actual Go-Live Date'] - df['Planned Go-Live Date']).dt.days
df['Project Duration'] = (df['Actual Go-Live Date'] - df['Actual Project Start Date']).dt.days
df['Planned Duration'] = df['Planned Project Duration (Days)']

# Ageing of WIP projects based on planned and actual go-live dates
df['Ageing (Months)'] = (df['Actual Go-Live Date'].fillna(datetime.today()) - df['Actual Project Start Date']).dt.days / 30
df['Ageing Category'] = pd.cut(df['Ageing (Months)'], bins=[0, 6, 12, 24, 36, float('inf')],
                               labels=['< 6 Months', '6-12 Months', '1-2 Years', '2-3 Years', '> 3 Years'])

# Identify delayed projects based on the actual go-live date vs planned go-live date
df['Is Delayed'] = df['Go-Live Date Variance'].apply(lambda x: 'Delayed' if x > 0 else 'On Time')

# Breakdown by Application
app_breakdown = df.groupby('Application Name').agg({'Project Code': 'count', 'Project Value': 'sum'}).reset_index()

# Projects by Status
status_breakdown = df['Is Delayed'].value_counts().reset_index()
status_breakdown.columns = ['Status', 'Count']

# Vertical Head performance
vertical_performance = df.groupby('Vertical Head').agg({'Go-Live Date Variance': 'mean', 'Project Value': 'sum'}).reset_index()

# Function to generate all the plots
def generate_dashboard():
    # KPIs
    total_projects = len(df)
    delayed_projects = len(df[df['Is Delayed'] == 'Delayed'])
    total_revenue = df['Project Value'].sum()
    avg_project_duration = df['Project Duration'].mean()

    # Create KPI text elements
    kpi_text = f"""
    Total Projects: {total_projects}
    Delayed Projects: {delayed_projects}
    Total Revenue: ${total_revenue:,.2f}
    Average Project Duration: {avg_project_duration:.1f} days
    """

    # Visualization 1: Status of Projects (Delayed vs On Time)
    fig1, ax1 = plt.subplots()
    ax1.pie(status_breakdown['Count'], labels=status_breakdown['Status'], autopct='%1.1f%%', startangle=90)
    ax1.set_title('Project Status Breakdown (Delayed vs On Time)')

    # Visualization 2: Application-wise Project and Revenue Breakdown
    fig2, ax2 = plt.subplots()
    ax2.bar(app_breakdown['Application Name'], app_breakdown['Project Code'], label='Projects')
    ax2.set_title('Revenue and Project Breakdown by Application')
    ax2.set_xlabel('Application Name')
    ax2.set_ylabel('Count of Projects')

    # Visualization 3: Vertical Head Performance
    fig3, ax3 = plt.subplots()
    ax3.bar(vertical_performance['Vertical Head'], vertical_performance['Project Value'], label='Revenue')
    ax3.set_title('Vertical Head Performance (Revenue)')
    ax3.set_xlabel('Vertical Head')
    ax3.set_ylabel('Total Revenue')

    # Visualization 4: WIP Projects by Ageing
    fig4, ax4 = plt.subplots()
    df['Ageing Category'].value_counts().plot(kind='bar', ax=ax4)
    ax4.set_title('Ageing Breakdown of WIP Projects')
    ax4.set_xlabel('Ageing Categories')
    ax4.set_ylabel('Number of Projects')

    # Show the plots
    plt.tight_layout()
    return kpi_text, fig1, fig2, fig3, fig4

# Gradio Interface for Dashboard with Layout
dashboard_interface = gr.Interface(fn=generate_dashboard, 
                                   inputs=[], 
                                   outputs=[gr.Textbox(), gr.Plot(), gr.Plot(), gr.Plot(), gr.Plot()],
                                   live=True, 
                                   description="Interactive Dashboard for Project Performance Analysis")

# Launch the dashboard
dashboard_interface.launch()


C:\Users\atul\AppData\Local\Temp\ipykernel_3408\2886506654.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Actual Go-Live Date'] = pd.to_datetime(df['Actual Go-Live Date'], errors='coerce')


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
